# Generate the data for "mutation matrices"

Both for codons (e.g. ATG, TGC) and amino acids (e.g. M, C).

**NOTE 1:** Because this boils down to calling [`AlignedSegment.get_aligned_pairs()`](https://pysam.readthedocs.io/en/latest/api.html#pysam.AlignedSegment.get_aligned_pairs) once for every read aligned to the selected MAGs, **this notebook is currently pretty slow**! I've optimized things to the point where this notebook takes around 20 hours on our cluster.

If necessary / desired it should be possible to speed this up even more, using stuff like parallelization / writing this in a faster language like C / etc. There may also be methods in pysam I've overlooked that will help do this faster (from what I can tell the only way to do this in pysam involves calling `get_aligned_pairs()`, but maybe I've missed something...)

**NOTE 2:** This doesn't actually generate the figures for the matrices -- it just outputs JSON files to a folder named `matrix-jsons/`, and another notebook will generate the figures based on these JSON files. This is intended to make it easier to regenerate the figures using different styles / etc. without having to wait hours for this stuff to finish.

**NOTE 3:** We're redoing things so that the overall matrices are only based on individual MAGs -- so it should be possible to either parallelize this on the level of each MAGs (could probably parallelize things even further), or just only run this for individual MAGs. HOWEVER: for now, I'm going to keep generating this data for all of the three selected MAGs, because (i) we need this data for the Syn/NonSyn and Non-nonsense / Nonsense barplots, and (ii) I don't really want to rewrite this entire notebook since this is currently a one-off analysis...

In [1]:
%run "Header.ipynb"
%run "GeneUtils.ipynb"

In [2]:
import copy
import time
import json
import pysam
import skbio
from collections import defaultdict, Counter
from statistics import mean
from parse_sco import parse_sco

## Go through all reads aligned to each genome and figure out which genes they intersect and which codons in these genes they fully cover

__This is the main bottleneck of this notebook__, at least as of writing.

Define a dict which we'll use to keep track of aligned codon frequencies for each codon, for each gene, for each genome.

- For each read, see which predicted genes (if any) this read intersects within the genome. Note that "intersects" doesn't mean "fully covers".

- For each of these genes, see which codons (if any) this read fully covers within the gene.

- Increment aligned codon frequencies for all codons accordingly.

The reason we do things this way, as opposed to iterating over just the reads overlapping each codon in each gene, is that doing things that way is really slow! I'm pretty sure it's because "find out which reads overlap this region" is a pretty slow operation when working with large datasets -- and also since these are long reads, doing this on the level of each codon means we're effectively doing a lot of redundant work (you can imagine that, for a given codon, the odds are pretty good that most reads overlapping it will also overlap adjacent codon(s)).

In [3]:
# There's probably a fancier way of generating this list, but this is fine.
codons = []
# Also, we figure out the reverse complements of each of the 64 3-mers in advance -- this avoids
# us having to call str(skbio.DNA(c).reverse_complement()) every time we see a codon, and saves a tiny
# amount of time per read (the skbio approach took ~9e-5 seconds every time; the new approach takes ~9e-7
# seconds every time). Considering we're going through well over a million reads, the time savings comes out
# to ... 130.977 seconds, aka 2 minutes 10 seconds, if I'm computing this correctly. So, not much, but it's
# something!
codon2revcomp = {}
nts = "ACGT"
for i in nts:
    for j in nts:
        for k in nts:
            c = "{}{}{}".format(i, j, k)
            codons.append(c)
            codon2revcomp[c] = str(skbio.DNA(c).reverse_complement())

In [4]:
# Maps sequence IDs to genes (keyed by their Index in the .sco file) to codons (keyed by (0-indexed!)
# left end, i.e. the lower of the two positional boundaries of the codon, regardless of if its gene
# is on the + or - strand) to observed aligned codon frequencies (keyed by just the triplet, e.g. "AAA").
#
# Example:
# {"edge_6104":                                Sequence
#     {1:                                      Gene index in the .sco file
#         {265:                                Left codon position
#             {"TTA": 1000, "TTT": 1, ... }    Aligned codon frequencies for this particular codon
#         }
#     }
# }
seq2gene2codon2alignedcodons = {}

In [ ]:
bf = pysam.AlignmentFile("../main-workflow/output/fully-filtered-and-sorted-aln.bam", "rb")

tT1 = time.time()
for seq in SEQS:
    df = parse_sco("../seqs/genes/{}.sco".format(seq))
    
    # We don't actually store any results in this, but we do use it for a slight optimization
    gene2isrev = {}
    
    # Initialize some of the data structures
    # NOTE: this is kind of slow. However, it still finishes within a few seconds, so not the most
    # important thing to optimize
    seq2gene2codon2alignedcodons[seq] = {}
    for gene_data in df.itertuples():
        
        validate_gene_coords(gene_data)        
        seq2gene2codon2alignedcodons[seq][gene_data.Index] = {}
        gene2isrev[gene_data.Index] = (gene_data.Strand == "-")
        
        codon_positions = get_gene_left_codon_positions(gene_data)

        # For each codon in this gene, keep track of all the codons spanning it from the various
        # alignments to this genome.
        for cpleft in codon_positions:
            seq2gene2codon2alignedcodons[seq][gene_data.Index][cpleft] = defaultdict(int)
            
    print("Finished initialization for seq = {}".format(seq))
    readtimes = []
    
    # Note that this isn't really a "read" so much as it is an aligned linear segment (and a read
    # can have multiple such segments derived from it, as discussed above).
    for ri, read in enumerate(bf.fetch(seq), 1):
        
        t1 = time.time()
        
        # Find all genes that this read intersects in this genome
        
        # These are 0-indexed coordinates (and segright is offset to the right by one; see
        # https://pysam.readthedocs.io/en/latest/api.html#pysam.AlignedSegment.reference_end)
        segleft = read.reference_start
        segright = read.reference_end
        
        if segleft is None or segright is None:
            raise ValueError("Read {} is unmapped? This shouldn't happen!".format(seg.query_name))
        
        if segleft >= segright:
            raise ValueError("Read {}'s coordinates in pysam seem messed up: left = {}, right = {}".format(
                seg.query_name, segleft, segright
            ))
        
        # Convert aligned segment boundaries to 1-indexed coordinates to make comparing with gene
        # coordinates from the .sco file easier.
        # Since segright was already offset to the right by 1, we don't need to do anything for it
        # (the gene coordinates are exact: a gene from [266, 712] starts at base 266 and ends at base 712,
        # using 1-indexing. So in order to make the read boundaries match, we'd add 1 for segright and then
        # subract 1 since segright was already 1 base off -- and n + 1 - 1 = n. (...math is hard)
        segleft += 1
        
        # Use vectorization to find genes overlapping this read: see https://stackoverflow.com/a/17071908
        # for details on why parentheses, etc., and
        # https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6
        # for justification on why this is useful (tldr: makes code go fast)
        genes_overlapping_read = list(
            df.loc[(df["RightEnd"] >= segleft) & (df["LeftEnd"] <= segright)].itertuples()
        )
        # Note about the above thing that just happened: you may be shaking your fist and saying "wait
        # itertuples is slow!" And yeah, kinda. But for whatever reason I've tried multiple times to keep
        # genes_overlapping_read as a DataFrame (and then later vectorize stuff like checking that a given
        # aligned pair covers a codon within the genes, etc) and the overhead costs seem to slow things down.
        # I am sure it's possible to speed things up more, but right now things seem good enough.

        # (Debugging code)
        # print("{} genes overlap read {}".format(len(genes_overlapping_read), ri))
        # print("Read {}, which ranges from {} to {}, overlaps these genes:".format(ri, segleft, segright))
        # print(genes_overlapping_read)
                
        # If no genes overlap this read, we are free to move on to the next read.
        if len(genes_overlapping_read) > 0:
            
            # Computing this is relatively slow, which is why we jump through so many hoops before we do this.
            # Each entry in get_aligned_pairs() is a tuple with 2 elements:
            # the first is the read pos and the second is the reference pos.
            # TODO: would it be possible to only do this for certain positions we care about? get_aligned_pairs()
            # returns a lot of stuff we don't need, e.g. regions of the read that don't intersect with any genes.
            ap = read.get_aligned_pairs(matches_only=True)
            
            # Doesn't look like getting this in advance saves much time, but I don't think it hurts.
            read_seq = read.query_sequence
            
            # We only consider the leftmost position of each codon, so we don't need to bother checking the last
            # two pairs of positions (since neither could be the leftmost position of a codon that this read
            # fully covers).
            for api, pair1 in enumerate(ap[:-2]):

                # Convert to 1-indexed position for ease of comparison with gene coordinates
                pair1_refpos = pair1[1] + 1
                  
                havent_checked_next_pairs = True
                for gene_data in genes_overlapping_read:
                    gl = gene_data.LeftEnd
                    
                    # Check that this pair is located within this gene and is the leftmost position of a
                    # codon in the gene. (Note that check works for both + or - strand genes.
                    # Whether the leftmost position is the "start" [i.e. CP 1] or "end" [i.e. CP 3] of
                    # the gene changes with the strand of the gene, but we'll account for that later on
                    # when we reverse-complement the codon if needed.)                  
                    if pair1_refpos >= gl and pair1_refpos <= gene_data.RightEnd - 2 and ((pair1_refpos - gl) % 3 == 0):
                        
                        # Nice! Looks like this read fully covers this codon.
                        
                        # If we haven't yet, check that this read doesn't skip over parts of the codon,
                        # or stuff like that. The reason this check is located *here* (and not
                        # before we loop over the genes) is that it seems like this is a faster strategy:
                        # only run these checks once we KNOW that this pair looks like it fully covers a
                        # codon, since many pairs might not meet that criteria.
                        #
                        # (And by recording that we've run this check once, in havent_checked_next_pairs,
                        # we can save the time cost of running the check multiple times.)
                        #
                        # I feel like an insane person trying to optimize this so much lmao.
                        if havent_checked_next_pairs:
                            # Check that the pairs are all consecutive (i.e. no "jumps" in the read,
                            # and no "jumps" in the reference)
                            # Since we don't consider the last two pairs in ap, pair2 and pair3 should
                            # always be available.
                            pair2 = ap[api + 1]
                            pair3 = ap[api + 2]

                            # For an aligned pair, [0] is the read pos and [1] is the reference pos.
                            # Result of caching this is probably negligible but... may as well
                            p10 = pair1[0]
                            p20 = pair2[0]
                            readpos_consecutive = p20 == (p10 + 1) and pair3[0] == (p20 + 1)
                            if not readpos_consecutive:
                                break

                            # (pair1_refpos is already off by 1 so no need to redo the addition operation)
                            p21 = pair2[1]
                            refpos_consecutive = p21 == pair1_refpos and pair3[1] == (p21 + 1)
                            if not refpos_consecutive:
                                break
                            havent_checked_next_pairs = False
                        
                        # Figure out what the read actually *says* in the alignment here.
                        # (It'll probably be a complete match most of the time, but there will
                        # be some occasional mismatches -- and seeing those is ... the whole point
                        # of this notebook.)

                        # We make sure to index the read by read coords, not reference coords!
                        aligned_codon = read_seq[p10: p10 + 3]

                        # Finally, update information about codon frequencies.
                        gi = gene_data.Index
                        if gene2isrev[gi]:
                            seq2gene2codon2alignedcodons[seq][gi][pair1_refpos][codon2revcomp[aligned_codon]] += 1
                        else:
                            seq2gene2codon2alignedcodons[seq][gi][pair1_refpos][aligned_codon] += 1

        t2 = time.time()
        readtimes.append(t2 - t1)
        if ri % 100 == 0:
            print("Seen {} reads so far in {}.".format(ri, seq))
            print("Total time taken thus far:          {} sec.".format(t2 - tT1))
            print("Average time per read for this seq: {} sec.".format(mean(readtimes)))

    # At this point, we've seen all the reads aligned to all the codons in this genome.
    # We can now "call" mutations based on the frequencies we've counted.
tT2 = time.time()

print("Figuring all that out took a total of {} seconds.".format(tT2 - tT1))

bf.close()
with open("matrix-jsons/seq2gene2codon2alignedcodons.json", "w") as dumpster:
    dumpster.write(json.dumps(seq2gene2codon2alignedcodons))

Finished initialization for seq = edge_6104
Seen 100 reads so far in edge_6104.
Total time taken thus far:          5.248497724533081 sec.
Average time per read for this seq: 0.045910487174987795 sec.
Seen 200 reads so far in edge_6104.
Total time taken thus far:          9.151857852935791 sec.
Average time per read for this seq: 0.042450501918792724 sec.
Seen 300 reads so far in edge_6104.
Total time taken thus far:          13.857287645339966 sec.
Average time per read for this seq: 0.043963789145151776 sec.
Seen 400 reads so far in edge_6104.
Total time taken thus far:          17.93490219116211 sec.
Average time per read for this seq: 0.043157677054405215 sec.
Seen 500 reads so far in edge_6104.
Total time taken thus far:          22.0102961063385 sec.
Average time per read for this seq: 0.04266665124893188 sec.
Seen 600 reads so far in edge_6104.
Total time taken thus far:          26.052297115325928 sec.
Average time per read for this seq: 0.042283867597579954 sec.
Seen 700 reads

Seen 5300 reads so far in edge_6104.
Total time taken thus far:          207.01934480667114 sec.
Average time per read for this seq: 0.03887722946562857 sec.
Seen 5400 reads so far in edge_6104.
Total time taken thus far:          211.16039872169495 sec.
Average time per read for this seq: 0.03892273717456394 sec.
Seen 5500 reads so far in edge_6104.
Total time taken thus far:          215.16939449310303 sec.
Average time per read for this seq: 0.03894257367740978 sec.
Seen 5600 reads so far in edge_6104.
Total time taken thus far:          219.5927197933197 sec.
Average time per read for this seq: 0.03903548470565251 sec.
Seen 5700 reads so far in edge_6104.
Total time taken thus far:          223.42464566230774 sec.
Average time per read for this seq: 0.03902157252294976 sec.
Seen 5800 reads so far in edge_6104.
Total time taken thus far:          227.21795010566711 sec.
Average time per read for this seq: 0.039001624584198 sec.
Seen 5900 reads so far in edge_6104.
Total time taken t

Seen 10500 reads so far in edge_6104.
Total time taken thus far:          396.55847930908203 sec.
Average time per read for this seq: 0.03762896083650135 sec.
Seen 10600 reads so far in edge_6104.
Total time taken thus far:          401.07324266433716 sec.
Average time per read for this seq: 0.03769850928828401 sec.
Seen 10700 reads so far in edge_6104.
Total time taken thus far:          405.45156478881836 sec.
Average time per read for this seq: 0.0377539833906655 sec.
Seen 10800 reads so far in edge_6104.
Total time taken thus far:          409.4919652938843 sec.
Average time per read for this seq: 0.03777724698737816 sec.
Seen 10900 reads so far in edge_6104.
Total time taken thus far:          413.6923830509186 sec.
Average time per read for this seq: 0.037814780309659624 sec.
Seen 11000 reads so far in edge_6104.
Total time taken thus far:          417.730087518692 sec.
Average time per read for this seq: 0.03783677649497986 sec.
Seen 11100 reads so far in edge_6104.
Total time t

Seen 15700 reads so far in edge_6104.
Total time taken thus far:          634.8798260688782 sec.
Average time per read for this seq: 0.04029727129419898 sec.
Seen 15800 reads so far in edge_6104.
Total time taken thus far:          639.704687833786 sec.
Average time per read for this seq: 0.04034649598447582 sec.
Seen 15900 reads so far in edge_6104.
Total time taken thus far:          644.681722164154 sec.
Average time per read for this seq: 0.04040462880764367 sec.
Seen 16000 reads so far in edge_6104.
Total time taken thus far:          649.3193969726562 sec.
Average time per read for this seq: 0.040440902322530746 sec.
Seen 16100 reads so far in edge_6104.
Total time taken thus far:          653.9925401210785 sec.
Average time per read for this seq: 0.040478932798279 sec.
Seen 16200 reads so far in edge_6104.
Total time taken thus far:          658.5889563560486 sec.
Average time per read for this seq: 0.040511805378360514 sec.
Seen 16300 reads so far in edge_6104.
Total time taken

Seen 20900 reads so far in edge_6104.
Total time taken thus far:          919.2304182052612 sec.
Average time per read for this seq: 0.043829896256113735 sec.
Seen 21000 reads so far in edge_6104.
Total time taken thus far:          924.4653072357178 sec.
Average time per read for this seq: 0.04386950402032761 sec.
Seen 21100 reads so far in edge_6104.
Total time taken thus far:          929.3187472820282 sec.
Average time per read for this seq: 0.04389064295032013 sec.
Seen 21200 reads so far in edge_6104.
Total time taken thus far:          933.9735934734344 sec.
Average time per read for this seq: 0.04390218451337994 sec.
Seen 21300 reads so far in edge_6104.
Total time taken thus far:          939.1061089038849 sec.
Average time per read for this seq: 0.04393602989089321 sec.
Seen 21400 reads so far in edge_6104.
Total time taken thus far:          944.4447777271271 sec.
Average time per read for this seq: 0.04397915305378281 sec.
Seen 21500 reads so far in edge_6104.
Total time ta

...

Seen 278100 reads so far in edge_2358.
Total time taken thus far:          38618.75628781319 sec.
Average time per read for this seq: 0.033779175526716006 sec.
Seen 278200 reads so far in edge_2358.
Total time taken thus far:          38621.239931344986 sec.
Average time per read for this seq: 0.0337753215922802 sec.
Seen 278300 reads so far in edge_2358.
Total time taken thus far:          38623.63229370117 sec.
Average time per read for this seq: 0.03377113108407068 sec.
Seen 278400 reads so far in edge_2358.
Total time taken thus far:          38625.73782944679 sec.
Average time per read for this seq: 0.033765923608651105 sec.
Seen 278500 reads so far in edge_2358.
Total time taken thus far:          38627.889108181 sec.
Average time per read for this seq: 0.033760889044783915 sec.
Seen 278600 reads so far in edge_2358.
Total time taken thus far:          38630.01285648346 sec.
Average time per read for this seq: 0.0337557649441272 sec.
Seen 278700 reads so far in edge_2358.
Total t

Seen 283300 reads so far in edge_2358.
Total time taken thus far:          38744.71852993965 sec.
Average time per read for this seq: 0.0335704823243841 sec.
Seen 283400 reads so far in edge_2358.
Total time taken thus far:          38747.86742568016 sec.
Average time per read for this seq: 0.0335691114092849 sec.
Seen 283500 reads so far in edge_2358.
Total time taken thus far:          38751.01125717163 sec.
Average time per read for this seq: 0.03356773135691513 sec.
Seen 283600 reads so far in edge_2358.
Total time taken thus far:          38754.22434592247 sec.
Average time per read for this seq: 0.03356656985407319 sec.
Seen 283700 reads so far in edge_2358.
Total time taken thus far:          38757.660730838776 sec.
Average time per read for this seq: 0.03356618265892193 sec.
Seen 283800 reads so far in edge_2358.
Total time taken thus far:          38760.90856862068 sec.
Average time per read for this seq: 0.033565114321550744 sec.
Seen 283900 reads so far in edge_2358.
Total t

Seen 288500 reads so far in edge_2358.
Total time taken thus far:          38942.61449074745 sec.
Average time per read for this seq: 0.03361710457909252 sec.
Seen 288600 reads so far in edge_2358.
Total time taken thus far:          38947.081869363785 sec.
Average time per read for this seq: 0.033620267324949916 sec.
Seen 288700 reads so far in edge_2358.
Total time taken thus far:          38951.49485516548 sec.
Average time per read for this seq: 0.03362323805251087 sec.
Seen 288800 reads so far in edge_2358.
Total time taken thus far:          38955.96430230141 sec.
Average time per read for this seq: 0.033626410611118304 sec.
Seen 288900 reads so far in edge_2358.
Total time taken thus far:          38960.16774392128 sec.
Average time per read for this seq: 0.03362864424176346 sec.
Seen 289000 reads so far in edge_2358.
Total time taken thus far:          38963.94772076607 sec.
Average time per read for this seq: 0.033629434695293337 sec.
Seen 289100 reads so far in edge_2358.
Tot

Seen 293700 reads so far in edge_2358.
Total time taken thus far:          39122.28212308884 sec.
Average time per read for this seq: 0.033599567193338814 sec.
Seen 293800 reads so far in edge_2358.
Total time taken thus far:          39125.445029973984 sec.
Average time per read for this seq: 0.033598224979586304 sec.
Seen 293900 reads so far in edge_2358.
Total time taken thus far:          39128.6865940094 sec.
Average time per read for this seq: 0.0335971558244101 sec.
Seen 294000 reads so far in edge_2358.
Total time taken thus far:          39131.94077587128 sec.
Average time per read for this seq: 0.033596133760043556 sec.
Seen 294100 reads so far in edge_2358.
Total time taken thus far:          39135.0997800827 sec.
Average time per read for this seq: 0.03359477803480945 sec.
Seen 294200 reads so far in edge_2358.
Total time taken thus far:          39138.220341444016 sec.
Average time per read for this seq: 0.0335932890360733 sec.
Seen 294300 reads so far in edge_2358.
Total 

Seen 298900 reads so far in edge_2358.
Total time taken thus far:          39263.5468480587 sec.
Average time per read for this seq: 0.03345391200593021 sec.
Seen 299000 reads so far in edge_2358.
Total time taken thus far:          39266.45666241646 sec.
Average time per read for this seq: 0.03345178865748504 sec.
Seen 299100 reads so far in edge_2358.
Total time taken thus far:          39269.20794010162 sec.
Average time per read for this seq: 0.03344911873759095 sec.
Seen 299200 reads so far in edge_2358.
Total time taken thus far:          39271.952031612396 sec.
Average time per read for this seq: 0.033446435855831055 sec.
Seen 299300 reads so far in edge_2358.
Total time taken thus far:          39274.9847111702 sec.
Average time per read for this seq: 0.03344472848170185 sec.
Seen 299400 reads so far in edge_2358.
Total time taken thus far:          39277.825670957565 sec.
Average time per read for this seq: 0.03344237286883668 sec.
Seen 299500 reads so far in edge_2358.
Total 

## Using the information we just computed for each genome, "call" mutations and store this information in the frequency data structures we set up earlier

This section is kind of slow, but on the order of "takes a few minutes on the cluster" and not on the order of "takes literally hours to run". We could make it more efficient if desired; most of my energy on optimization here thus far has been spent on the earlier step in this notebook.

Note that the main output of the above section (`seq2gene2codon2alignedcodons`) has already been written out to a JSON file -- in practice, it's useful to be able to start execution again from this bottom section after stopping the above section (e.g. if something goes wrong _here_, then we don't have to rerun the top part of this notebook again). Even if the entire notebook is run in a single shot, we still save and then load the `seq2gene2codon2alignedcodons` object anyway.

### Initialize data structures that we'll store frequency data in

In [4]:
# 64x63 dict: each key is a triplet of {A, C, G, T}, and each value is another dict with all the other codons
codon2codon2freq = {}

# 21x20 dict: each key is a proteinogenic amino acid (A, C, D, E, F, ...), limited to just
# stuff in the standard genetic code (i.e. ignoring selenocystine and pyrrolsine) but including
# "*", representing a stop codon.
aa2aa2freq = {}

# 64-key dict: maps each triplet to an integer indicating how frequently this triplet occurs in all genes
# in the genomes (i.e. not counting mutations into this triplet).
codon2freq = {}

# 21-key dict: maps amino acid/stop codon to integer indicating frequency across all genes.
aa2freq = {}

aas = set([])
for c in codons:
    aas.add(str(skbio.DNA(c).translate()))
    
# Initialize dicts to 0s
for c1 in codons:
    codon2codon2freq[c1] = {c2: 0 for c2 in set(codons) - set([c1])}
    codon2freq[c1] = 0
    
for aa1 in aas:
    aa2aa2freq[aa1] = {aa2: 0 for aa2 in set(aas) - set([aa1])}
    aa2freq[aa1] = 0

In [5]:
with open("matrix-jsons/seq2gene2codon2alignedcodons.json", "r") as loadster:
    seq2gene2codon2alignedcodons = json.load(loadster)

In [7]:
for seq in SEQS:
    fasta = skbio.DNA.read("../seqs/{}.fasta".format(seq))
    df = parse_sco("../seqs/genes/{}.sco".format(seq))
    for gene_data in df.itertuples():
        print("On gene {} in seq {}.".format(gene_data.Index, seq))
        for cpleft in range(gene_data.LeftEnd, gene_data.RightEnd + 1, 3):
            
            # Make note of the codon sequence and amino acid encoded by this codon in the "reference" genome.
            # (Keep in mind that the gene data in the .sco file uses 1-indexed coords, so we need to convert
            # accordingly.)
            codon_dna = fasta[cpleft - 1: cpleft + 2]
            if gene_data.Strand == "-":
                codon_dna = codon_dna.reverse_complement()

            codon_seq = str(codon_dna)
            aa = str(codon_dna.translate())
            
            # Update frequencies accordingly.
            codon2freq[codon_seq] += 1
            aa2freq[aa] += 1
            
            # We can finally compute stats re: number of mismatching and matching codons.
            aligned_codons = seq2gene2codon2alignedcodons[seq][str(gene_data.Index)][str(cpleft)]
            
            # Ignore weird, low-coverage cases. Copying from the SynAndNonsense barplots notebook, which
            # uses similar logic:
            #  If the reference codon sequence isn't even included in the aligned codons (but if
            #  there are still other codons included in the alignment???), we're likely
            #  at a weird low-coverage portion of the alignment. Ignore these cases (there are 116 such
            #  problematic codons as of writing this, mostly in edge_1671 -- something might be wrong with
            #  the alignment, which I had to recover from a backup recently -- will rerun and check).
            #
            #  This implicitly accounts for the case where 0 codons are present in aligned_codons, as well.
            if codon_seq in aligned_codons:
                
                num_aligned_codons = sum(aligned_codons.values())
                alt_codon_frac = (num_aligned_codons - aligned_codons[codon_seq]) / num_aligned_codons

                # print("sum of vals of ac is {}".format(sum(aligned_codons.values())))
                # print("Codon {} from {} to {} in gene {} in seq {} has mutations: {}".format(
                #     codon_seq, cpleft, cpleft + 2, gene_data.Index, seq, aligned_codons
                # ))

                # Using minfreq = 0.5%
                if alt_codon_frac > 0.005:

                    # Subset aligned_codons to just the alternate codons. I guess we could also just use "del".
                    alt_codons = {c: aligned_codons[c] for c in aligned_codons if c != codon_seq}

                    # Retrieve max-freq alternate codon.
                    # Based on https://stackoverflow.com/a/280156.
                    # (Note that if there's a tie, the result is arbitrary. Shouldn't be a big deal. Making note
                    # of in the paper.)
                    max_freq_alt_codon = max(alt_codons, key=alt_codons.get)
                    codon2codon2freq[codon_seq][max_freq_alt_codon] += 1

                    # print("Is mutation! And max freq alt codon is {}".format(max_freq_alt_codon))

                    # NOTE: I guess you could argue that we should do this another way, where we actually compute
                    # the translations of all the alt codons and then pick the most common AA/stop codon from there?
                    #
                    # You could argue this either way: doing it based on just the mutated codon keeps the matrices
                    # consistent and lessens the impact of small errors, while taking into account all alt codon
                    # translations could help show weird things where multiple mutations have similar consequences.
                    # Hmm.
                    #
                    # TODO: think about!
                    alt_codon_aa = str(skbio.DNA(max_freq_alt_codon).translate())
                    if alt_codon_aa != aa:
                        aa2aa2freq[aa][alt_codon_aa] += 1
                        # print("Is nonsyn mutation! Alt {} codes for {}; orig coded for {}".format(
                        #     max_freq_alt_codon, alt_codon_aa, aa
                        # ))

# Write out stuff for further analysis / in case of crisis
with open("matrix-jsons/codon2codon2freq.json", "w") as dumpster:
    dumpster.write(json.dumps(codon2codon2freq))
    
with open("matrix-jsons/codon2freq.json", "w") as dumpster:
    dumpster.write(json.dumps(codon2freq))
    
with open("matrix-jsons/aa2aa2freq.json", "w") as dumpster:
    dumpster.write(json.dumps(aa2aa2freq))

with open("matrix-jsons/aa2freq.json", "w") as dumpster:
    dumpster.write(json.dumps(aa2freq))

On gene 1 in seq edge_6104.
On gene 2 in seq edge_6104.
On gene 3 in seq edge_6104.
On gene 4 in seq edge_6104.
On gene 5 in seq edge_6104.
On gene 6 in seq edge_6104.
On gene 7 in seq edge_6104.
On gene 8 in seq edge_6104.
On gene 9 in seq edge_6104.
On gene 10 in seq edge_6104.
On gene 11 in seq edge_6104.
On gene 12 in seq edge_6104.
On gene 13 in seq edge_6104.
On gene 14 in seq edge_6104.
On gene 15 in seq edge_6104.
On gene 16 in seq edge_6104.
On gene 17 in seq edge_6104.
On gene 18 in seq edge_6104.
On gene 19 in seq edge_6104.
On gene 20 in seq edge_6104.
On gene 21 in seq edge_6104.
On gene 22 in seq edge_6104.
On gene 23 in seq edge_6104.
On gene 24 in seq edge_6104.
On gene 25 in seq edge_6104.
On gene 26 in seq edge_6104.
On gene 27 in seq edge_6104.
On gene 28 in seq edge_6104.
On gene 29 in seq edge_6104.
On gene 30 in seq edge_6104.
On gene 31 in seq edge_6104.
On gene 32 in seq edge_6104.
On gene 33 in seq edge_6104.
On gene 34 in seq edge_6104.
On gene 35 in seq edge_

On gene 286 in seq edge_6104.
On gene 287 in seq edge_6104.
On gene 288 in seq edge_6104.
On gene 289 in seq edge_6104.
On gene 290 in seq edge_6104.
On gene 291 in seq edge_6104.
On gene 292 in seq edge_6104.
On gene 293 in seq edge_6104.
On gene 294 in seq edge_6104.
On gene 295 in seq edge_6104.
On gene 296 in seq edge_6104.
On gene 297 in seq edge_6104.
On gene 298 in seq edge_6104.
On gene 299 in seq edge_6104.
On gene 300 in seq edge_6104.
On gene 301 in seq edge_6104.
On gene 302 in seq edge_6104.
On gene 303 in seq edge_6104.
On gene 304 in seq edge_6104.
On gene 305 in seq edge_6104.
On gene 306 in seq edge_6104.
On gene 307 in seq edge_6104.
On gene 308 in seq edge_6104.
On gene 309 in seq edge_6104.
On gene 310 in seq edge_6104.
On gene 311 in seq edge_6104.
On gene 312 in seq edge_6104.
On gene 313 in seq edge_6104.
On gene 314 in seq edge_6104.
On gene 315 in seq edge_6104.
On gene 316 in seq edge_6104.
On gene 317 in seq edge_6104.
On gene 318 in seq edge_6104.
On gene 31

On gene 569 in seq edge_6104.
On gene 570 in seq edge_6104.
On gene 571 in seq edge_6104.
On gene 572 in seq edge_6104.
On gene 573 in seq edge_6104.
On gene 574 in seq edge_6104.
On gene 575 in seq edge_6104.
On gene 576 in seq edge_6104.
On gene 577 in seq edge_6104.
On gene 578 in seq edge_6104.
On gene 579 in seq edge_6104.
On gene 580 in seq edge_6104.
On gene 581 in seq edge_6104.
On gene 582 in seq edge_6104.
On gene 583 in seq edge_6104.
On gene 584 in seq edge_6104.
On gene 585 in seq edge_6104.
On gene 586 in seq edge_6104.
On gene 587 in seq edge_6104.
On gene 588 in seq edge_6104.
On gene 589 in seq edge_6104.
On gene 590 in seq edge_6104.
On gene 591 in seq edge_6104.
On gene 592 in seq edge_6104.
On gene 593 in seq edge_6104.
On gene 594 in seq edge_6104.
On gene 595 in seq edge_6104.
On gene 596 in seq edge_6104.
On gene 597 in seq edge_6104.
On gene 598 in seq edge_6104.
On gene 599 in seq edge_6104.
On gene 600 in seq edge_6104.
On gene 601 in seq edge_6104.
On gene 60

On gene 847 in seq edge_6104.
On gene 848 in seq edge_6104.
On gene 849 in seq edge_6104.
On gene 850 in seq edge_6104.
On gene 851 in seq edge_6104.
On gene 852 in seq edge_6104.
On gene 853 in seq edge_6104.
On gene 854 in seq edge_6104.
On gene 855 in seq edge_6104.
On gene 856 in seq edge_6104.
On gene 857 in seq edge_6104.
On gene 858 in seq edge_6104.
On gene 859 in seq edge_6104.
On gene 860 in seq edge_6104.
On gene 861 in seq edge_6104.
On gene 862 in seq edge_6104.
On gene 863 in seq edge_6104.
On gene 864 in seq edge_6104.
On gene 865 in seq edge_6104.
On gene 866 in seq edge_6104.
On gene 867 in seq edge_6104.
On gene 868 in seq edge_6104.
On gene 869 in seq edge_6104.
On gene 870 in seq edge_6104.
On gene 871 in seq edge_6104.
On gene 872 in seq edge_6104.
On gene 873 in seq edge_6104.
On gene 874 in seq edge_6104.
On gene 875 in seq edge_6104.
On gene 876 in seq edge_6104.
On gene 877 in seq edge_6104.
On gene 878 in seq edge_6104.
On gene 879 in seq edge_6104.
On gene 88

On gene 1126 in seq edge_6104.
On gene 1127 in seq edge_6104.
On gene 1128 in seq edge_6104.
On gene 1129 in seq edge_6104.
On gene 1130 in seq edge_6104.
On gene 1131 in seq edge_6104.
On gene 1132 in seq edge_6104.
On gene 1133 in seq edge_6104.
On gene 1134 in seq edge_6104.
On gene 1135 in seq edge_6104.
On gene 1136 in seq edge_6104.
On gene 1137 in seq edge_6104.
On gene 1138 in seq edge_6104.
On gene 1139 in seq edge_6104.
On gene 1140 in seq edge_6104.
On gene 1141 in seq edge_6104.
On gene 1142 in seq edge_6104.
On gene 1143 in seq edge_6104.
On gene 1144 in seq edge_6104.
On gene 1145 in seq edge_6104.
On gene 1146 in seq edge_6104.
On gene 1147 in seq edge_6104.
On gene 1148 in seq edge_6104.
On gene 1149 in seq edge_6104.
On gene 1150 in seq edge_6104.
On gene 1151 in seq edge_6104.
On gene 1152 in seq edge_6104.
On gene 1153 in seq edge_6104.
On gene 1154 in seq edge_6104.
On gene 1155 in seq edge_6104.
On gene 1156 in seq edge_6104.
On gene 1157 in seq edge_6104.
On gene 

...

On gene 1357 in seq edge_2358.
On gene 1358 in seq edge_2358.
On gene 1359 in seq edge_2358.
On gene 1360 in seq edge_2358.
On gene 1361 in seq edge_2358.
On gene 1362 in seq edge_2358.
On gene 1363 in seq edge_2358.
On gene 1364 in seq edge_2358.
On gene 1365 in seq edge_2358.
On gene 1366 in seq edge_2358.
On gene 1367 in seq edge_2358.
On gene 1368 in seq edge_2358.
On gene 1369 in seq edge_2358.
On gene 1370 in seq edge_2358.
On gene 1371 in seq edge_2358.
On gene 1372 in seq edge_2358.
On gene 1373 in seq edge_2358.
On gene 1374 in seq edge_2358.
On gene 1375 in seq edge_2358.
On gene 1376 in seq edge_2358.
On gene 1377 in seq edge_2358.
On gene 1378 in seq edge_2358.
On gene 1379 in seq edge_2358.
On gene 1380 in seq edge_2358.
On gene 1381 in seq edge_2358.
On gene 1382 in seq edge_2358.
On gene 1383 in seq edge_2358.
On gene 1384 in seq edge_2358.
On gene 1385 in seq edge_2358.
On gene 1386 in seq edge_2358.
On gene 1387 in seq edge_2358.
On gene 1388 in seq edge_2358.
On gene 

On gene 1637 in seq edge_2358.
On gene 1638 in seq edge_2358.
On gene 1639 in seq edge_2358.
On gene 1640 in seq edge_2358.
On gene 1641 in seq edge_2358.
On gene 1642 in seq edge_2358.
On gene 1643 in seq edge_2358.
On gene 1644 in seq edge_2358.
On gene 1645 in seq edge_2358.
On gene 1646 in seq edge_2358.
On gene 1647 in seq edge_2358.
On gene 1648 in seq edge_2358.
On gene 1649 in seq edge_2358.
On gene 1650 in seq edge_2358.
On gene 1651 in seq edge_2358.
On gene 1652 in seq edge_2358.
On gene 1653 in seq edge_2358.
On gene 1654 in seq edge_2358.
On gene 1655 in seq edge_2358.
On gene 1656 in seq edge_2358.
On gene 1657 in seq edge_2358.
On gene 1658 in seq edge_2358.
On gene 1659 in seq edge_2358.
On gene 1660 in seq edge_2358.
On gene 1661 in seq edge_2358.
On gene 1662 in seq edge_2358.
On gene 1663 in seq edge_2358.
On gene 1664 in seq edge_2358.
On gene 1665 in seq edge_2358.
On gene 1666 in seq edge_2358.
On gene 1667 in seq edge_2358.
On gene 1668 in seq edge_2358.
On gene 

On gene 1905 in seq edge_2358.
On gene 1906 in seq edge_2358.
On gene 1907 in seq edge_2358.
On gene 1908 in seq edge_2358.
On gene 1909 in seq edge_2358.
On gene 1910 in seq edge_2358.
On gene 1911 in seq edge_2358.
On gene 1912 in seq edge_2358.
On gene 1913 in seq edge_2358.
On gene 1914 in seq edge_2358.
On gene 1915 in seq edge_2358.
On gene 1916 in seq edge_2358.
On gene 1917 in seq edge_2358.
On gene 1918 in seq edge_2358.
On gene 1919 in seq edge_2358.
On gene 1920 in seq edge_2358.
On gene 1921 in seq edge_2358.
On gene 1922 in seq edge_2358.
On gene 1923 in seq edge_2358.
On gene 1924 in seq edge_2358.
On gene 1925 in seq edge_2358.
On gene 1926 in seq edge_2358.
On gene 1927 in seq edge_2358.
On gene 1928 in seq edge_2358.
On gene 1929 in seq edge_2358.
On gene 1930 in seq edge_2358.
On gene 1931 in seq edge_2358.
On gene 1932 in seq edge_2358.
On gene 1933 in seq edge_2358.
On gene 1934 in seq edge_2358.
On gene 1935 in seq edge_2358.
On gene 1936 in seq edge_2358.
On gene 

On gene 2177 in seq edge_2358.
On gene 2178 in seq edge_2358.
On gene 2179 in seq edge_2358.
On gene 2180 in seq edge_2358.
On gene 2181 in seq edge_2358.
On gene 2182 in seq edge_2358.
On gene 2183 in seq edge_2358.
On gene 2184 in seq edge_2358.
On gene 2185 in seq edge_2358.
On gene 2186 in seq edge_2358.
On gene 2187 in seq edge_2358.
On gene 2188 in seq edge_2358.
On gene 2189 in seq edge_2358.
On gene 2190 in seq edge_2358.
On gene 2191 in seq edge_2358.
On gene 2192 in seq edge_2358.
On gene 2193 in seq edge_2358.
On gene 2194 in seq edge_2358.
On gene 2195 in seq edge_2358.
On gene 2196 in seq edge_2358.
On gene 2197 in seq edge_2358.
On gene 2198 in seq edge_2358.
On gene 2199 in seq edge_2358.
On gene 2200 in seq edge_2358.
On gene 2201 in seq edge_2358.
On gene 2202 in seq edge_2358.
On gene 2203 in seq edge_2358.
On gene 2204 in seq edge_2358.
On gene 2205 in seq edge_2358.
On gene 2206 in seq edge_2358.
On gene 2207 in seq edge_2358.
On gene 2208 in seq edge_2358.
On gene 

On gene 2451 in seq edge_2358.
On gene 2452 in seq edge_2358.
On gene 2453 in seq edge_2358.
On gene 2454 in seq edge_2358.
On gene 2455 in seq edge_2358.
On gene 2456 in seq edge_2358.
On gene 2457 in seq edge_2358.
On gene 2458 in seq edge_2358.
On gene 2459 in seq edge_2358.
On gene 2460 in seq edge_2358.
On gene 2461 in seq edge_2358.
On gene 2462 in seq edge_2358.
On gene 2463 in seq edge_2358.
On gene 2464 in seq edge_2358.
On gene 2465 in seq edge_2358.
On gene 2466 in seq edge_2358.
On gene 2467 in seq edge_2358.
On gene 2468 in seq edge_2358.
On gene 2469 in seq edge_2358.
On gene 2470 in seq edge_2358.
On gene 2471 in seq edge_2358.
On gene 2472 in seq edge_2358.
On gene 2473 in seq edge_2358.
On gene 2474 in seq edge_2358.
On gene 2475 in seq edge_2358.
On gene 2476 in seq edge_2358.
On gene 2477 in seq edge_2358.
On gene 2478 in seq edge_2358.
On gene 2479 in seq edge_2358.
On gene 2480 in seq edge_2358.
On gene 2481 in seq edge_2358.
On gene 2482 in seq edge_2358.
On gene 